### NYSE + NASDAQ progressions
- take in txt with meta data 
- find Symbols that make up 80% of Total market Cap 
- ensure symbols are formatted for YFinance API 
- get daily OHLCV data from Yfinance for each, save in SQL db

In [1]:
import pandas as pd
import numpy as np
import re

def load_stock_data(file_path):
    """Load stock data from CSV file."""
    try:
        df = pd.read_csv(file_path)
        print(f"Successfully loaded data from {file_path}")
        return df
    except Exception as e:
        print(f"Error loading file: {e}")
        return None

def filter_columns(df):
    """Keep only Symbol and Market Cap columns."""
    if "Symbol" in df.columns and "Market Cap" in df.columns:
        filtered_df = df[["Symbol", "Market Cap"]].copy()
        # Remove any rows with NaN values
        filtered_df = filtered_df.dropna()
        
        # Ensure Market Cap is numeric
        filtered_df["Market Cap"] = pd.to_numeric(filtered_df["Market Cap"], errors="coerce")
        filtered_df = filtered_df.dropna()
        
        # Calculate statistics
        total_symbols = filtered_df.shape[0]
        total_market_cap = filtered_df["Market Cap"].sum()
        
        print(f"Found {total_symbols} unique symbols with a total market cap of ${total_market_cap:,.2f}")
        return filtered_df
    else:
        print("Error: Required columns 'Symbol' and 'Market Cap' not found in the data")
        return None

def apply_pareto_principle(df, top_percent_market_cap=80):
    """
    Find the top X symbols that make up Y% of the total market cap.
    Returns the filtered DataFrame and the percentage of symbols this represents.
    """
    # Sort by Market Cap in descending order
    sorted_df = df.sort_values("Market Cap", ascending=False).reset_index(drop=True)
    
    # Calculate cumulative percentage of market cap
    total_market_cap = sorted_df["Market Cap"].sum()
    sorted_df["Cumulative Market Cap"] = sorted_df["Market Cap"].cumsum()
    sorted_df["Cumulative Percentage"] = (sorted_df["Cumulative Market Cap"] / total_market_cap) * 100
    
    # Find where we hit the target percentage
    target_df = sorted_df[sorted_df["Cumulative Percentage"] <= top_percent_market_cap]
    
    # If empty (happens if first row already exceeds the percentage), take at least one row
    if target_df.empty:
        target_df = sorted_df.iloc[:1]
    else:
        # Add one more row to ensure we're at or above the target percentage
        next_index = target_df.index[-1] + 1
        if next_index < len(sorted_df):
            target_df = pd.concat([target_df, sorted_df.iloc[[next_index]]])
    
    symbols_percent = (len(target_df) / len(sorted_df)) * 100
    actual_market_cap_percent = target_df["Market Cap"].sum() / total_market_cap * 100
    
    print(f"Keeping {len(target_df)} symbols ({symbols_percent:.2f}% of total) that represent {actual_market_cap_percent:.2f}% of total market cap")
    
    return target_df

def get_ticker_list(df):
    """Extract ticker symbols from DataFrame."""
    return df["Symbol"].tolist()

def convert_tickers_for_yfinance(tickers):
    """Convert ticker symbols to yfinance format."""
    converted_tickers = []
    
    for ticker in tickers:
        # Convert to string in case it's not already
        ticker_str = str(ticker)
        # Replace '^', '/', and '.' with '-'
        converted = re.sub(r'[\^/\.]', '-', ticker_str)
        converted_tickers.append(converted)
    
    print(f"Converted {len(converted_tickers)} tickers to yfinance format")
    return converted_tickers

def clean_tickers(file_path, pareto_percent=99):
    # File path - you'll need to update this
    file_path = file_path 
    
    # Step 1: Load the CSV file
    df = load_stock_data(file_path)
    if df is None:
        return
    
    # Step 2: Filter columns
    filtered_df = filter_columns(df)
    if filtered_df is None:
        return
    
    # Step 3: Apply Pareto principle (adjust the percentage as needed)
    target_market_cap_percent = pareto_percent  # Target percentage of market cap to capture
    pareto_df = apply_pareto_principle(filtered_df, target_market_cap_percent)
    
    # Step 4: Get list of tickers
    tickers = get_ticker_list(pareto_df)
    print(f"Selected tickers: {tickers[:5]}{'...' if len(tickers) > 5 else ''}")
    
    # Step 5: Convert tickers for yfinance
    yfinance_tickers = convert_tickers_for_yfinance(tickers)
    
    # You can now use these tickers with yfinance
    print("Ready to use with yfinance!")
    
    return yfinance_tickers

# Get tickers for both exchanges using the clean_tickers function
NYSE_tickers = clean_tickers(file_path="NYSE.csv", pareto_percent=60)
NASDAQ_tickers = clean_tickers(file_path="NASDAQ.csv", pareto_percent=60) 


Successfully loaded data from NYSE.csv
Found 2358 unique symbols with a total market cap of $41,402,108,200,657.00
Keeping 126 symbols (5.34% of total) that represent 60.14% of total market cap
Selected tickers: ['BRK/B', 'BRK/A', 'TSM', 'LLY', 'WMT']...
Converted 126 tickers to yfinance format
Ready to use with yfinance!
Successfully loaded data from NASDAQ.csv
Found 3863 unique symbols with a total market cap of $31,488,945,564,668.00
Keeping 17 symbols (0.44% of total) that represent 60.62% of total market cap
Selected tickers: ['AAPL', 'MSFT', 'NVDA', 'AMZN', 'GOOG']...
Converted 17 tickers to yfinance format
Ready to use with yfinance!


In [2]:
import os
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import sqlite3
import time
from tqdm import tqdm
import logging

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("stock_data_download.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

def create_database(db_path="stock_data.db"):
    """Create SQLite database with tables for stock data."""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Create table for daily stock data
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS stock_daily_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            symbol TEXT NOT NULL,
            date TEXT NOT NULL,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            volume INTEGER,
            UNIQUE(symbol, date)
        )
        ''')
        
        # Create index for faster queries
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_symbol_date ON stock_daily_data (symbol, date)')
        
        # Create table for ticker metadata
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS ticker_metadata (
            symbol TEXT PRIMARY KEY,
            source TEXT,
            last_updated TEXT,
            status TEXT
        )
        ''')
        
        conn.commit()
        logger.info(f"Database created/connected successfully at {db_path}")
        return conn
    except Exception as e:
        logger.error(f"Error creating database: {e}")
        return None

def download_stock_data(tickers, conn, start_date='2020-01-01', end_date=None, 
                        interval='1d', source='combined', chunk_size=50):
    """
    Download stock data for multiple tickers and save to SQLite database.
    
    Args:
        tickers: List of ticker symbols
        conn: SQLite connection
        start_date: Start date for data download
        end_date: End date for data download (defaults to today)
        interval: Data interval ('1d', '1wk', etc.)
        source: Source of tickers for metadata
        chunk_size: Number of tickers to download at once
    """
    if end_date is None:
        end_date = datetime.now().strftime('%Y-%m-%d')
    
    # Parse dates
    start_dt = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date)
    today = pd.to_datetime(datetime.now().date())
    
    if end_dt > today:
        logger.info(f"End date {end_dt.date()} is in the future; adjusting to today's date {today.date()}.")
        end_dt = today
    
    # Process tickers in chunks to avoid API limits
    total_tickers = len(tickers)
    successful_downloads = 0
    failed_downloads = 0
    
    logger.info(f"Starting download of {total_tickers} tickers from {start_date} to {end_date}")
    
    # Process in chunks
    for i in range(0, total_tickers, chunk_size):
        chunk = tickers[i:i+chunk_size]
        logger.info(f"Processing chunk {i//chunk_size + 1}/{(total_tickers-1)//chunk_size + 1} ({len(chunk)} tickers)")
        
        try:
            # Download data for chunk
            df = yf.download(
                tickers=chunk,
                start=start_dt.strftime('%Y-%m-%d'),
                end=(end_dt + timedelta(days=1)).strftime('%Y-%m-%d'),
                interval=interval,
                group_by='ticker',
                auto_adjust=True,
                prepost=False,
                threads=True,
                progress=False
            )
            
            # Check if we got data
            if df.empty:
                logger.warning(f"No data returned for chunk {i//chunk_size + 1}")
                failed_downloads += len(chunk)
                continue
                
            # Process each ticker in the chunk
            for ticker in chunk:
                try:
                    # For a single ticker, the dataframe is not multi-indexed
                    if len(chunk) == 1:
                        ticker_df = df.copy()
                    else:
                        # For multiple tickers, extract the specific ticker data
                        ticker_df = df[ticker].copy()
                    
                    # Skip if no data for this ticker
                    if ticker_df.empty:
                        logger.warning(f"No data for {ticker}")
                        failed_downloads += 1
                        
                        # Update metadata table with failed status
                        cursor = conn.cursor()
                        cursor.execute('''
                        INSERT OR REPLACE INTO ticker_metadata (symbol, source, last_updated, status)
                        VALUES (?, ?, ?, ?)
                        ''', (ticker, source, datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'FAILED'))
                        conn.commit()
                        continue
                    
                    # Reset index to make date a column
                    ticker_df = ticker_df.reset_index()
                    
                    # Ensure we have the expected columns
                    required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
                    
                    # Check and rename columns if needed
                    if 'Datetime' in ticker_df.columns:
                        ticker_df = ticker_df.rename(columns={'Datetime': 'Date'})
                    
                    # Ensure all required columns exist
                    missing_columns = [col for col in required_columns if col not in ticker_df.columns]
                    if missing_columns:
                        logger.warning(f"Missing columns for {ticker}: {missing_columns}")
                        failed_downloads += 1
                        continue
                    
                    # Convert date to string format for SQLite
                    ticker_df['Date'] = ticker_df['Date'].dt.strftime('%Y-%m-%d')
                    
                    # Insert data into database
                    cursor = conn.cursor()
                    
                    # Prepare data for insertion
                    data_to_insert = []
                    for _, row in ticker_df.iterrows():
                        data_to_insert.append((
                            ticker,
                            row['Date'],
                            row['Open'],
                            row['High'],
                            row['Low'],
                            row['Close'],
                            row['Volume']
                        ))
                    
                    # Use executemany for better performance
                    cursor.executemany('''
                    INSERT OR REPLACE INTO stock_daily_data 
                    (symbol, date, open, high, low, close, volume)
                    VALUES (?, ?, ?, ?, ?, ?, ?)
                    ''', data_to_insert)
                    
                    # Update metadata
                    cursor.execute('''
                    INSERT OR REPLACE INTO ticker_metadata (symbol, source, last_updated, status)
                    VALUES (?, ?, ?, ?)
                    ''', (ticker, source, datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'SUCCESS'))
                    
                    conn.commit()
                    successful_downloads += 1
                    
                except Exception as e:
                    logger.error(f"Error processing {ticker}: {e}")
                    failed_downloads += 1
                    
                    # Update metadata with error status
                    try:
                        cursor = conn.cursor()
                        cursor.execute('''
                        INSERT OR REPLACE INTO ticker_metadata (symbol, source, last_updated, status)
                        VALUES (?, ?, ?, ?)
                        ''', (ticker, source, datetime.now().strftime('%Y-%m-%d %H:%M:%S'), f'ERROR: {str(e)[:100]}'))
                        conn.commit()
                    except Exception as inner_e:
                        logger.error(f"Error updating metadata for {ticker}: {inner_e}")
            
            # Sleep between chunks to avoid rate limiting
            if i + chunk_size < total_tickers:
                logger.info(f"Sleeping for 2 seconds between chunks...")
                time.sleep(2)
                
        except Exception as e:
            logger.error(f"Error downloading chunk {i//chunk_size + 1}: {e}")
            failed_downloads += len(chunk)
    
    logger.info(f"Download complete. Successful: {successful_downloads}, Failed: {failed_downloads}")
    return successful_downloads, failed_downloads

def get_database_stats(conn):
    """Get statistics about the database."""
    try:
        cursor = conn.cursor()
        
        # Get total number of records
        cursor.execute("SELECT COUNT(*) FROM stock_daily_data")
        total_records = cursor.fetchone()[0]
        
        # Get number of unique symbols
        cursor.execute("SELECT COUNT(DISTINCT symbol) FROM stock_daily_data")
        unique_symbols = cursor.fetchone()[0]
        
        # Get date range
        cursor.execute("SELECT MIN(date), MAX(date) FROM stock_daily_data")
        date_range = cursor.fetchone()
        
        # Get success/failure counts
        cursor.execute("SELECT status, COUNT(*) FROM ticker_metadata GROUP BY status")
        status_counts = cursor.fetchall()
        
        logger.info(f"Database Statistics:")
        logger.info(f"Total records: {total_records}")
        logger.info(f"Unique symbols: {unique_symbols}")
        logger.info(f"Date range: {date_range[0]} to {date_range[1]}")
        logger.info(f"Status counts: {status_counts}")
        
        return {
            "total_records": total_records,
            "unique_symbols": unique_symbols,
            "date_range": date_range,
            "status_counts": status_counts
        }
    except Exception as e:
        logger.error(f"Error getting database stats: {e}")
        return None

def GetData(start_date, end_date):
    # Combine ticker lists from your previous code
    # Assuming NYSE_tickers and NASDAQ_tickers are already defined from your previous code
    combined_tickers = list(set(NYSE_tickers + NASDAQ_tickers))  # Remove duplicates
    logger.info(f"Combined {len(NYSE_tickers)} NYSE tickers and {len(NASDAQ_tickers)} NASDAQ tickers into {len(combined_tickers)} unique tickers")
    
    # Create database connection
    db_path = "stock_market_data.db"
    conn = create_database(db_path)
    
    if conn is None:
        logger.error("Failed to create database connection. Exiting.")
        return
    
    try:
        # Set date range - adjust as needed
        start_date = start_date
        end_date = end_date

        # Download data
        successful, failed = download_stock_data(
            tickers=combined_tickers,
            conn=conn,
            start_date=start_date,
            end_date=end_date,
            interval='1d',
            source='combined',
            chunk_size=500  # Adjust based on API limits
        )
        
        # Get database statistics
        stats = get_database_stats(conn)
        
        logger.info(f"Data download complete. Database saved to {db_path}")
        logger.info(f"Downloaded data for {successful} tickers successfully. Failed for {failed} tickers.")
        
    except Exception as e:
        logger.error(f"Error in main process: {e}")
    finally:
        # Close connection
        if conn:
            conn.close()
            logger.info("Database connection closed")

#Update Database 
start_date = '2024-01-01'  # 5 years of data
end_date = datetime.now().strftime('%Y-%m-%d')  # Today
GetData(start_date=start_date, end_date=end_date)


2025-04-21 20:51:11,169 - INFO - Combined 126 NYSE tickers and 17 NASDAQ tickers into 143 unique tickers
2025-04-21 20:51:11,170 - INFO - Database created/connected successfully at stock_market_data.db
2025-04-21 20:51:11,172 - INFO - Starting download of 143 tickers from 2024-01-01 to 2025-04-21
2025-04-21 20:51:11,172 - INFO - Processing chunk 1/1 (143 tickers)
2025-04-21 20:51:15,228 - INFO - Download complete. Successful: 143, Failed: 0
2025-04-21 20:51:15,478 - INFO - Database Statistics:
2025-04-21 20:51:15,478 - INFO - Total records: 2392952
2025-04-21 20:51:15,479 - INFO - Unique symbols: 1552
2025-04-21 20:51:15,479 - INFO - Date range: 2019-01-02 to 2025-04-21
2025-04-21 20:51:15,479 - INFO - Status counts: [('SUCCESS', 1552)]
2025-04-21 20:51:15,480 - INFO - Data download complete. Database saved to stock_market_data.db
2025-04-21 20:51:15,480 - INFO - Downloaded data for 143 tickers successfully. Failed for 0 tickers.
2025-04-21 20:51:15,481 - INFO - Database connection clo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from datetime import datetime, timedelta
import logging

# Set up logging if not already done
if not logging.getLogger().hasHandlers():
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler("ad_analysis.log"),
            logging.StreamHandler()
        ]
    )
logger = logging.getLogger()

def get_database_date_range(db_path="stock_market_data.db"):
    """
    Get the date range available in the database
    """
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Get the min and max dates from the database
        cursor.execute("SELECT MIN(date), MAX(date) FROM stock_daily_data")
        min_date, max_date = cursor.fetchone()
        
        conn.close()
        
        return pd.to_datetime(min_date), pd.to_datetime(max_date)
    
    except Exception as e:
        logger.error(f"Error getting database date range: {e}")
        # Fallback to a reasonable default
        today = datetime.now()
        return today - timedelta(days=365*2), today

def calculate_ad_data_from_db(db_path="stock_market_data.db"):
    """
    Calculate Advance-Decline data using all tickers in our SQL database
    
    Parameters:
    db_path (str): Path to the SQLite database
    
    Returns:
    pd.DataFrame: DataFrame with A/D data
    """
    logger.info(f"Calculating A/D data for all tickers in the database")
    
    # Get date range from database
    start_date, end_date = get_database_date_range(db_path)
    logger.info(f"Using date range from database: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
    
    try:
        conn = sqlite3.connect(db_path)
        
        # Get all unique dates in the database
        date_query = "SELECT DISTINCT date FROM stock_daily_data ORDER BY date"
        all_dates = pd.read_sql_query(date_query, conn)['date']
        
        # Initialize DataFrame for A/D data
        ad_data = pd.DataFrame(index=all_dates)
        ad_data.index = pd.to_datetime(ad_data.index)
        
        # Initialize columns
        ad_data['Advances'] = 0
        ad_data['Declines'] = 0
        ad_data['Unchanged'] = 0
        
        # Get list of all tickers in the database
        ticker_query = "SELECT DISTINCT symbol FROM stock_daily_data"
        all_tickers = pd.read_sql_query(ticker_query, conn)['symbol'].tolist()
        
        logger.info(f"Processing {len(all_tickers)} tickers")
        
        # Process each ticker
        for ticker in all_tickers:
            # Query the database for this ticker's data
            query = f"""
            SELECT date, close 
            FROM stock_daily_data 
            WHERE symbol = '{ticker}' 
            ORDER BY date
            """
            
            df = pd.read_sql_query(query, conn)
            
            # Skip if no data or only one data point
            if len(df) <= 1:
                continue
                
            # Calculate returns
            df['return'] = df['close'].pct_change()
            
            # Skip the first row (NaN return)
            df = df.iloc[1:]
            
            # Classify each day's return
            for _, row in df.iterrows():
                date = row['date']
                ret = row['return']
                
                if pd.to_datetime(date) in ad_data.index:
                    if ret > 0:
                        ad_data.loc[pd.to_datetime(date), 'Advances'] += 1
                    elif ret < 0:
                        ad_data.loc[pd.to_datetime(date), 'Declines'] += 1
                    else:
                        ad_data.loc[pd.to_datetime(date), 'Unchanged'] += 1
        
        # Close the connection
        conn.close()
        
        # Calculate Daily A/D and Cumulative A/D
        ad_data['Daily_AD'] = ad_data['Advances'] - ad_data['Declines']
        ad_data['Cumulative_AD'] = ad_data['Daily_AD'].cumsum()
        
        logger.info(f"Successfully calculated A/D data")
        return ad_data
    
    except Exception as e:
        logger.error(f"Error calculating A/D data: {e}")
        return pd.DataFrame()  # Return empty DataFrame on error

def get_spy_data_from_db(db_path="stock_market_data.db"):
    """
    Get SPY data from the database
    """
    try:
        # Get date range from database
        start_date, end_date = get_database_date_range(db_path)
        
        conn = sqlite3.connect(db_path)
        
        # Query the database for SPY data
        query = f"""
        SELECT date, close 
        FROM stock_daily_data 
        WHERE symbol = 'SPY' 
        ORDER BY date
        """
        
        spy = pd.read_sql_query(query, conn)
        conn.close()
        
        # Convert date to datetime
        spy['date'] = pd.to_datetime(spy['date'])
        spy = spy.set_index('date')
        
        if len(spy) == 0:
            raise ValueError("SPY data not found in database")
        
        return spy
    
    except Exception as e:
        logger.error(f"Error getting SPY data from database: {e}")
        # If we can't get SPY from the database, fall back to yfinance
        logger.info("Falling back to yfinance for SPY data")
        import yfinance as yf
        
        # Use the database date range if available
        try:
            start_date, end_date = get_database_date_range(db_path)
            spy = yf.download('SPY', start=start_date, end=end_date + timedelta(days=1))
        except:
            # If all else fails, get 2 years of data
            end_date = datetime.now()
            start_date = end_date - timedelta(days=365*2)
            spy = yf.download('SPY', start=start_date, end=end_date)
        
        return spy[['Close']].rename(columns={'Close': 'close'})

def plot_ad_analysis(db_path="stock_market_data.db"):
    """
    Calculate and plot A/D analysis for all tickers in the database
    """
    # Calculate A/D data
    ad_data = calculate_ad_data_from_db(db_path)
    
    # Check if we have data
    if len(ad_data) == 0:
        logger.error("Failed to calculate A/D data")
        return None
    
    # Calculate 90-day MA of Cumulative A/D
    ad_data['Cumulative_AD_90MA'] = ad_data['Cumulative_AD'].rolling(window=30).mean()
    
    # Calculate red background indicator
    ad_data['Red_Background'] = (ad_data['Cumulative_AD'] < ad_data['Cumulative_AD_90MA']).astype(int)
    
    # Get SPY data
    spy = get_spy_data_from_db(db_path)
    
    # Align SPY data with our A/D data
    common_dates = ad_data.index.intersection(spy.index)
    ad_data = ad_data.loc[common_dates]
    spy = spy.loc[common_dates]
    
    # Plot the results
    plt.figure(figsize=(15, 10))
    
    # Plot 1: SPY price
    plt.subplot(3, 1, 1)
    plt.plot(spy.index, spy['close'], 'k-')
    plt.title('SPY Price')
    plt.grid(True, alpha=0.3)
    
    # Plot 2: A/D Line with 90-day MA
    plt.subplot(3, 1, 2)
    plt.plot(ad_data.index, ad_data['Cumulative_AD'], 'b-', label='Cumulative A/D')
    plt.plot(ad_data.index, ad_data['Cumulative_AD_90MA'], 'r-', label='90-day MA')
    plt.title('Market-Wide Advance-Decline Line')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 3: SPY with Red Background
    plt.subplot(3, 1, 3)
    plt.plot(spy.index, spy['close'], 'k-')
    
    # Highlight red background periods - fixed version
    red_periods = []
    current_start = None
    
    for i in range(len(ad_data)):
        is_red = ad_data['Red_Background'].iloc[i] == 1
        
        if is_red and current_start is None:
            current_start = ad_data.index[i]
        elif not is_red and current_start is not None:
            red_periods.append((current_start, ad_data.index[i]))
            current_start = None
    
    # Handle case where we end on a red period
    if current_start is not None:
        red_periods.append((current_start, ad_data.index[-1]))
    
    # Apply the red background for each period
    for start, end in red_periods:
        plt.axvspan(start, end, alpha=0.2, color='red')
    
    plt.title('SPY with Red Background Indicator')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Return the data for further analysis if needed
    return {
        'ad_data': ad_data,
        'spy': spy
    }

def calculate_market_statistics(results):
    """
    Calculate additional market statistics from the A/D analysis
    """
    if results is None or 'ad_data' not in results or 'spy' not in results:
        logger.error("Invalid results for market statistics calculation")
        return None
    
    ad_data = results['ad_data']
    spy = results['spy']
    
    # Make sure the indexes are aligned
    common_dates = ad_data.index.intersection(spy.index)
    ad_aligned = ad_data.loc[common_dates]
    spy_aligned = spy.loc[common_dates]
    
    # Calculate correlation between SPY and A/D line
    correlation = spy_aligned['close'].corr(ad_aligned['Cumulative_AD'])
    
    # Calculate percentage of days with red background
    red_days_pct = ad_aligned['Red_Background'].mean() * 100
    
    # Calculate SPY returns
    spy_returns = spy_aligned['close'].pct_change().dropna()
    
    # Get corresponding red background values
    red_background = ad_aligned['Red_Background'].loc[spy_returns.index]
    
    # Separate returns for red and non-red periods
    red_returns = spy_returns[red_background == 1]
    non_red_returns = spy_returns[red_background == 0]
    
    # Calculate statistics
    avg_red_return = red_returns.mean() * 100 if len(red_returns) > 0 else 0
    avg_non_red_return = non_red_returns.mean() * 100 if len(non_red_returns) > 0 else 0
    
    # Calculate cumulative return during red and non-red periods
    cum_red_return = ((1 + red_returns).prod() - 1) * 100 if len(red_returns) > 0 else 0
    cum_non_red_return = ((1 + non_red_returns).prod() - 1) * 100 if len(non_red_returns) > 0 else 0
    
    # Print statistics
    print("\nMarket Statistics:")
    print(f"Correlation between SPY and A/D Line: {correlation:.4f}")
    print(f"Percentage of days with red background: {red_days_pct:.2f}%")
    print(f"Average daily SPY return during red periods: {avg_red_return:.4f}%")
    print(f"Average daily SPY return during non-red periods: {avg_non_red_return:.4f}%")
    print(f"Cumulative SPY return during red periods: {cum_red_return:.2f}%")
    print(f"Cumulative SPY return during non-red periods: {cum_non_red_return:.2f}%")
    
    return {
        'correlation': correlation,
        'red_days_pct': red_days_pct,
        'avg_red_return': avg_red_return,
        'avg_non_red_return': avg_non_red_return,
        'cum_red_return': cum_red_return,
        'cum_non_red_return': cum_non_red_return
    }

# Example usage
if __name__ == "__main__":
    # Database path
    db_path = "stock_market_data.db"
    
    # Run the analysis and plot
    results = plot_ad_analysis(db_path)
    
    # Calculate and display market statistics
    if results:
        stats = calculate_market_statistics(results)


2025-04-21 20:51:15,734 - INFO - Calculating A/D data for all tickers in the database
2025-04-21 20:51:15,867 - INFO - Using date range from database: 2019-01-02 to 2025-04-21
2025-04-21 20:51:16,576 - INFO - Processing 1552 tickers
/var/folders/0l/68dwwf7n4k14jjgryv9tt5mw0000gn/T/ipykernel_71144/4168156369.py:97: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['return'] = df['close'].pct_change()
/var/folders/0l/68dwwf7n4k14jjgryv9tt5mw0000gn/T/ipykernel_71144/4168156369.py:97: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['return'] = df['close'].pct_change()
/var/folders/0l/68dwwf7n4k14jjg